In [36]:
#!pip install jyquickhelper
from jyquickhelper import add_notebook_menu


#Pour les api et données requetables
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
add_notebook_menu()

## <center>1. Importation des données</center>

In [2]:
#Racine principale de récupération des données
glob_path = "https://data.enseignementsup-recherche.gouv.fr/api/records/1.0/search/?dataset="

### 1.2 Vœux de poursuite d'études et de réorientation dans l'enseignement supérieur et réponses des établissements



https://data.enseignementsup-recherche.gouv.fr/explore/dataset/fr-esr-parcoursup/information/

###### On a 12760 lignes d'individus et l'Api ne permet pas de récupérer plus de 10 000 lignes et donc on a troué comme astuce de récupérer selon le type de l'établissement avant de merger les bases


In [3]:
#Segmentation pour pouvoir tout récupérer car l'API n'autorise pas plus de 10 000 lignes 
type_etablissement = ["2_Lyc%C3%A9es", "1_Universit%C3%A9s", "3_Autres+formations"]


In [4]:
#Le lien de récupération
voeux_poursuite = glob_path+"fr-esr-parcoursup&rows=6020&sort=tri&facet=session&facet=contrat_etab&facet=cod_uai&facet=g_ea_lib_vx&facet=dep_lib&facet=region_etab_aff&facet=acad_mies&facet=select_form&facet=fili&facet=form_lib_voe_acc&facet=regr_forma&facet=fil_lib_voe_acc&facet=detail_forma&facet=tri&facet=cod_aff_form&q="


In [5]:
#Récupération selon le type d'établissement
data_etablissement = [requests.get(voeux_poursuite + type_etablissement[x]) for x in range(3)]

In [6]:
#Vérification
data_etablissement

[<Response [200]>, <Response [200]>, <Response [200]>]

In [7]:
#Conversion de json à pd.Dataframe
data_etablissement = [pd.json_normalize(x.json()['records'][:]) for x in data_etablissement ]

In [8]:
glob_data = pd.concat(data_etablissement)
glob_data.columns = [c.replace('fields.', '') for c in list(glob_data.columns) ]

In [10]:

# Import seaborn
import seaborn as sns
# Apply the default theme
sns.set_theme()
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
#sns.catplot(data=glob_data[:], kind="swarm", x="contrat_etab", y="acc_tot", hue="select_form")


In [166]:
#!pip install pandas-profiling --user


### 1.3 INSERTION PROFESSIONNEL



https://data.enseignementsup-recherche.gouv.fr/api/records/1.0/search/?dataset=fr-esr-insertion_professionnelle-lp&q=&rows=1500&facet=annee&facet=diplome&facet=numero_de_l_etablissement&facet=etablissement&facet=academie&facet=domaine&facet=code_de_la_discipline&facet=discipline&facet=situation&facet=cle_etab&facet=cle_disc&facet=id_paysage&refine.annee=2018

In [42]:
insertion = glob_path + "fr-esr-insertion_professionnelle-"
niveau = ["lp", "master"]
complement = "&q=&rows=1500&sort=-annee&facet=annee&facet=diplome&facet=numero_de_l_etablissement&facet=etablissement&facet=academie&facet=domaine&facet=code_de_la_discipline&facet=discipline&facet=situation&facet=cle_etab&facet=cle_disc&facet=id_paysage&refine.annee=2018"

In [43]:
#Récupération selon le niveau
data_insertion = [requests.get(insertion + niveau[x] + complement) for x in range(2)]

In [45]:
#Conversion de json à pd.Dataframe
data_insertion = [pd.json_normalize(x.json()['records'][:]) for x in data_insertion ]

In [ ]:
###Les données selon le niveau sont identifiées par la varibale datsetid

In [56]:
data_insertion[0].shape

(1432, 36)

In [57]:
data_insertion[1].shape

(1500, 36)

In [49]:
glob_data_insertion = pd.concat(data_insertion)
glob_data_insertion.columns = [c.replace('fields.', '') for c in list(glob_data_insertion.columns) ]

In [50]:
glob_data_insertion.head()

,datasetid,recordid,record_timestamp,discipline,salaire_brut_annuel_estime,salaire_net_mensuel_regional_3eme_quartile,salaire_net_mensuel_median_regional,cle_disc,taux_de_reponse,etablissementactuel,emplois_cadre_ou_professions_intermediaires,taux_dinsertion,emplois_exterieurs_a_la_region_de_luniversite,emplois_stables,diplome,salaire_net_median_des_emplois_a_temps_plein,poids_de_la_discipline,taux_de_chomage_regional,code_du_domaine,emplois_a_temps_plein,code_de_la_discipline,de_diplomes_boursiers,academie,numero_de_l_etablissement,domaine,nombre_de_reponses,femmes,salaire_net_mensuel_regional_1er_quartile,cle_etab,emplois_cadre,situation,annee,id_paysage,etablissement,code_de_l_academie,remarque
0,fr-esr-insertion_professionnelle-lp,da765b6a4d45dd6886a8baeba37997c653f36b41,2021-12-15T22:16:37.978000+00:00,"Ensemble formations juridiques, économiques et...",27000,2240,1860,disc01_18,78,Université Côte d'Azur (UCA),50,96,34,82,LICENCE PRO,1730,49,9,DEG,95,disc01,14,Nice,0060931E,"Droit, économie et gestion",91,60,1410,Nice - Sophia-Antipolis_18,7,18 mois après le diplôme,2018,7CYWd,Nice - Sophia-Antipolis,A23,NaN
1,fr-esr-insertion_professionnelle-lp,fd7af98d5194b196646e8a56d246429bfb6d3fd3,2021-12-15T22:16:37.978000+00:00,Économie,ns,2240,1860,disc03_18,81,Université Côte d'Azur (UCA),ns,90,ns,ns,LICENCE PRO,ns,16,9,DEG,ns,disc03,14,Nice,0060931E,"Droit, économie et gestion",29,ns,1410,Nice - Sophia-Antipolis_18,ns,18 mois après le diplôme,2018,7CYWd,Nice - Sophia-Antipolis,A23,NaN
2,fr-esr-insertion_professionnelle-lp,8e0659da8279539be416739ff8bf43a896b75cc4,2021-12-15T22:16:37.978000+00:00,Gestion,27900,2240,1860,disc04_18,75,Université Côte d'Azur (UCA),56,100,24,90,LICENCE PRO,1790,27,9,DEG,92,disc04,14,Nice,0060931E,"Droit, économie et gestion",49,51,1410,Nice - Sophia-Antipolis_18,6,18 mois après le diplôme,2018,7CYWd,Nice - Sophia-Antipolis,A23,NaN
3,fr-esr-insertion_professionnelle-lp,01e6b76cfc0190483488a5e2b810015a62d142d4,2021-12-15T22:16:37.978000+00:00,Gestion,28200,2240,1860,disc04_30,75,Université Côte d'Azur (UCA),54,98,24,94,LICENCE PRO,1810,27,9,DEG,96,disc04,14,Nice,0060931E,"Droit, économie et gestion",49,51,1410,Nice - Sophia-Antipolis_30,6,30 mois après le diplôme,2018,7CYWd,Nice - Sophia-Antipolis,A23,NaN
4,fr-esr-insertion_professionnelle-lp,8e7f79b211b5071c5c9faf34a25cc5e4d51b4bd7,2021-12-15T22:16:37.978000+00:00,"Lettres, langues, arts",ns,2240,1860,disc06_18,63,Université Côte d'Azur (UCA),ns,ns,ns,ns,LICENCE PRO,ns,1,9,LLA,ns,disc06,14,Nice,0060931E,"Lettres, langues, arts",3,ns,1410,Nice - Sophia-Antipolis_18,ns,18 mois après le diplôme,2018,7CYWd,Nice - Sophia-Antipolis,A23,NaN


# BROUILLON

In [75]:
columns_names = pd.json_normalize(req.json()['facet_groups'][:])

In [76]:
columns_names = list(ses_2020.columns)
columns_names1 = [c.replace('fields.', '') for c in columns_names ]

In [77]:
dic=dict()
for i in range(len(columns_names)) :
    dic[columns_names[i]] = columns_names1[i]
ses_2020.rename(dic)

,datasetid,recordid,record_timestamp,fields.nb_voe_pp_at,fields.nb_voe_pp_bp,fields.nb_voe_pp_bg,fields.pct_sansmention,fields.fil_lib_voe_acc,fields.acc_brs,fields.acc_bg,fields.tri,fields.acc_bg_mention,fields.session,fields.form_lib_voe_acc,fields.cod_aff_form,fields.fili,fields.lien_form_psup,fields.prop_tot_bp_brs,fields.acad_mies,fields.taux_adm_psup_gen,fields.contrat_etab,fields.pct_bp_mention,fields.ran_grp3,fields.acc_finpp,fields.acc_pc,fields.nb_cla_pp_bg,fields.acc_bt,fields.nb_cla_pp_bt_brs,fields.nb_voe_pp_bt,fields.prop_tot_bg_brs,fields.taux_adm_psup_pro,fields.pct_aca_orig,fields.prop_tot_bt,fields.acc_pp,fields.voe_tot,fields.nb_voe_pp,fields.lib_comp_voe_ins,fields.cod_uai,fields.pct_acc_debutpp,fields.dep,fields.prop_tot_bp,fields.pct_tb,fields.prop_tot_bt_brs,fields.pct_etab_orig,fields.pct_mention_nonrenseignee,fields.acc_ab,fields.acc_aca_orig,fields.g_ea_lib_vx,fields.pct_ab,fields.nb_voe_pc_bp,fields.pct_acc_finpp,fields.nb_voe_pp_bg_brs,fields.nb_cla_pp_bp,fields.ran_grp1,fields.pct_bg,fields.pct_bours,fields.nb_voe_pc_at,fields.acc_aca_orig_idf,fields.pct_f,fields.acc_bt_mention,fields.nb_cla_pp,fields.acc_term,fields.prop_tot,fields.nb_voe_pc_bt,fields.acc_datebac,fields.acc_tot,fields.region_etab_aff,fields.nb_cla_pp_bg_brs,fields.pct_bt,fields.prop_tot_bg,fields.nb_voe_pp_bt_brs,fields.acc_mention_nonrenseignee,fields.nb_voe_pp_bp_brs,fields.lib_grp2,fields.acc_bp_mention,fields.acc_sansmention,fields.acc_tb,fields.prop_tot_at,fields.capa_fin,fields.acc_at,fields.nb_cla_pp_at,fields.acc_tot_f,fields.lib_grp1,fields.voe_tot_f,fields.acc_neobac,fields.nb_voe_pc,fields.pct_bg_mention,fields.pct_bp,fields.acc_debutpp,fields.acc_term_f,fields.lib_grp3,fields.pct_neobac,fields.pct_aca_orig_idf,fields.nb_cla_pc,fields.nb_cla_pp_bp_brs,fields.pct_b,fields.g_olocalisation_des_formations,fields.taux_adm_psup,fields.pct_bt_mention,fields.nb_cla_pp_bt,fields.ran_grp2,fields.nb_voe_pc_bg,fields.dep_lib,fields.acc_bp,fields.select_form,fields.acc_b,fields.pct_acc_datebac,fields.taux_adm_psup_techno,geometry.type,geometry.coordinates,fields.detail_forma,fields.nb_cla_pp_internat,fields.nb_voe_pp_internat,fields.acc_internat,fields.nb_cla_pp_pasinternat
0,fr-esr-parcoursup,5ddd4059ccae7206ae2f39b37d08495258e52dff,2021-02-04T13:10:28.998000+00:00,21,19,4,30.769231,Métiers de l'eau,9,1,2_Lycées,0,2020,BTS - Production,23320,BTS,https://dossier.parcoursup.fr/Candidat/carte?A...,3.0,La Réunion,6.0,Public,15.38,4,13.0,1,3,10,38,96,2.0,13.0,92.31,37.0,15,145,140,Lycée de St Paul 4 - BTS - Production - Métier...,9741380G,12.50,974,4.0,0.000000,22.0,7.69,0.0,6,12,Lycée de St Paul 4,46.153846,3,81.25,2,12,41.0,7.692308,69.23,1,12,12.50,7,90,1,47,1,12.0,16,La Réunion,2,76.923077,3.0,57,0,13,Tous les candidats sauf les Bac technologiques...,2,4,0,3.0,15,3,8,2,Bacheliers technologiques toutes séries,50,13,5,0.00,15.384615,2.0,0,Bacheliers professionnels toutes séries,81.25,92.31,1,10,23.076923,"[-20.9794, 55.3165]",34.0,53.85,67,3,0,La Réunion,2,formation selective,3,75.00,81.0,Point,"[55.3165, -20.9794]",NaN,NaN,NaN,NaN,NaN
1,fr-esr-parcoursup,a7f51872796481d96bbfcc6cfcc1fd13a8a291c3,2021-02-04T13:10:28.998000+00:00,9,8,3,0.000000,"Analyse, conduite et stratégie de l'entreprise...",0,0,2_Lycées,0,2020,BTS - Agricole,23461,BTS,https://dossier.parcoursup.fr/Candidat/carte?A...,2.0,Clermont-Ferrand,0.0,Public,100.00,10,1.0,0,3,0,2,11,0.0,100.0,0.00,9.0,1,46,31,AgroSup Dijon Direction Enseignement à Distanc...,0632001H,100.00,63,8.0,100.000000,2.0,0,0.0,0,0,AgroSup Dijon Direction Enseignement à Distanc...,0.000000,2,100.00,0,6,4.0,0.000000,0.00,9,0,0.00,0,20,0,25,3,1.0,1,Auvergne-Rhône-Alpes,0,0.000000,3.0,4,0,2,Bacheliers professionnels toutes séries,1,0,1,5.0,20,0,3,0,Tous les candidats sauf les Bac technologiques...,23,1,15,0.00,100.000000,1.0,0,Bacheliers technologiques toutes séries,100.00,0.00,5,2,0.000000,"[45.7749, 3.1766]",65.0,0.00,8,6,1,Puy-de-Dôme,1,formation selective,0,100.00,0.0,Point,"[3.1766, 45.7749]",NaN,Na